# Defining a Fixed-Floating Interest Rate Swap

Simple introduction to the inputs required to define a Ibor deposit and a FRA

In [1]:
from financepy.utils import *
from financepy.products.rates import *

####################################################################
#  FINANCEPY BETA Version 0.350 - This build: 30 Apr 2024 at 22:35 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Creating the Swap

We begin by setting the market conventions we wish to use.

In [2]:
swap_cal_type = CalendarTypes.TARGET
bd_type = BusDayAdjustTypes.FOLLOWING
dg_type = DateGenRuleTypes.BACKWARD

We then define the Fixed Leg

In [3]:
fixed_cpn = 0.05
fixed_freq_type = FrequencyTypes.ANNUAL
fixed_dc_type = DayCountTypes.ACT_360

And then we define the Floating Leg

In [4]:
float_spread = 0.0
float_freq_type = FrequencyTypes.SEMI_ANNUAL
float_dc_type = DayCountTypes.ACT_360

We need to specify the size and direction of the swap.

In [5]:
swap_type = SwapTypes.RECEIVE
notional = 10 * ONE_MILLION

And then we define when the swap begins and ends.

In [6]:
start_dt = Date(20, 6, 2018)
maturity_dt = Date(20, 9, 2025)

We can now create the swap object.

In [7]:
swap = IborSwap(start_dt,
                    maturity_dt,
                    swap_type,
                    fixed_cpn,
                    fixed_freq_type,
                    fixed_dc_type,
                    notional,
                    float_spread,
                    float_freq_type,
                    float_dc_type,
                    swap_cal_type,
                    bd_type,
                    dg_type)

In [8]:
print(swap)

OBJECT TYPE: IborSwap
OBJECT TYPE: SwapFixedLeg
START DATE: 20-JUN-2018
TERMINATION DATE: 20-SEP-2025
MATURITY DATE: 22-SEP-2025
NOTIONAL: 10000000
PRINCIPAL: 0.0
LEG TYPE: SwapTypes.RECEIVE
COUPON: 0.05
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.TARGET
BUS DAY ADJUST: BusDayAdjustTypes.FOLLOWING
DATE GEN TYPE: DateGenRuleTypes.BACKWARD

OBJECT TYPE: SwapFloatLeg
START DATE: 20-JUN-2018
TERMINATION DATE: 20-SEP-2025
MATURITY DATE: 22-SEP-2025
NOTIONAL: 10000000
SWAP TYPE: SwapTypes.PAY
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360
CALENDAR: CalendarTypes.TARGET
BUS DAY ADJUST: BusDayAdjustTypes.FOLLOWING
DATE GEN TYPE: DateGenRuleTypes.BACKWARD



In [9]:
swap.print_payments()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS SCHEDULE:
+---------+-------------+-------------+-------------+-------+----------+------+-----------+
| PAY_NUM |    PAY_dt   |  ACCR_START |   ACCR_END  |  DAYS | YEARFRAC | RATE |    PMNT   |
+---------+-------------+-------------+-------------+-------+----------+------+-----------+
|    1    | 20-SEP-2018 | 20-JUN-2018 | 20-SEP-2018 |  92.0 |  0.2556  | 5.0  | 127777.78 |
|    2    | 20-SEP-2019 | 20-SEP-2018 | 20-SEP-2019 | 365.0 |  1.0139  | 5.0  | 506944.44 |
|    3    | 21-SEP-2020 | 20-SEP-2019 | 21-SEP-2020 | 367.0 |  1.0194  | 5.0  | 509722.22 |
|    4    | 20-SEP-2021 | 21-SEP-2020 | 20-SEP-2021 | 364.0 |  1.0111  | 5.0  | 505555.56 |
|    5    | 20-SEP-2022 | 20-SEP-2021 | 20-SEP-2022 | 365.0 |  1.0139  | 5.0  | 506944.44 |
|    6    | 20-SEP-2023 | 20-SEP-2022 | 20-SEP-2023 | 365.0 |  1.0139  | 5.0  | 506944.44 |
|    7    | 20-SEP-

## Valuation

In [10]:
value_dt = Date(20,3,2020)

In [11]:
settle_dt = value_dt.add_weekdays(0)

This is just using a flat curve without any Ibor accrual factors.

In [12]:
from financepy.market.curves.discount_curve_flat import DiscountCurveFlat

In [13]:
discount_curve = DiscountCurveFlat(value_dt, 0.05, FrequencyTypes.SEMI_ANNUAL)

Need to specify the next fixing on the floating leg

In [14]:
swap.value(settle_dt, discount_curve, discount_curve)

219755.211551575

We can examine the individual legs - here is the floating leg valuation

In [15]:
swap.print_float_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS VALUATION:
+---------+-------------+----------+--------+-----------+--------+-----------+------------+
| PAY_NUM |    PAY_dt   | NOTIONAL |  IBOR  |    PMNT   |   DF   |     PV    |   CUM_PV   |
+---------+-------------+----------+--------+-----------+--------+-----------+------------+
|    1    | 20-SEP-2018 | 10000000 |  0.0   |    0.0    |  0.0   |    0.0    |    0.0     |
|    2    | 20-MAR-2019 | 10000000 |  0.0   |    0.0    |  0.0   |    0.0    |    0.0     |
|    3    | 20-SEP-2019 | 10000000 |  0.0   |    0.0    |  0.0   |    0.0    |    0.0     |
|    4    | 20-MAR-2020 | 10000000 |  0.0   |    0.0    |  0.0   |    0.0    |    0.0     |
|    5    | 21-SEP-2020 | 10000000 | 5.0007 | 256979.26 | 0.9753 | 250643.82 | 250643.82  |
|    6    | 22-MAR-2021 | 10000000 | 4.9997 | 252764.41 | 0.9517 | 240545.23 | 491189.05  |
|    7    |

And here is the fixed leg valuation

In [16]:
swap.print_fixed_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS VALUATION:
+---------+-------------+----------+------+-----------+--------+-----------+------------+
| PAY_NUM |    PAY_dt   | NOTIONAL | RATE |    PMNT   |   DF   |     PV    |   CUM_PV   |
+---------+-------------+----------+------+-----------+--------+-----------+------------+
|    1    | 20-SEP-2018 | 10000000 | 5.0  | 127777.78 |  0.0   |    0.0    |    0.0     |
|    2    | 20-SEP-2019 | 10000000 | 5.0  | 506944.44 |  0.0   |    0.0    |    0.0     |
|    3    | 21-SEP-2020 | 10000000 | 5.0  | 509722.22 | 0.9753 | 497155.79 | 497155.79  |
|    4    | 20-SEP-2021 | 10000000 | 5.0  | 505555.56 | 0.9285 | 469413.12 | 966568.91  |
|    5    | 20-SEP-2022 | 10000000 | 5.0  | 506944.44 | 0.8838 | 448021.63 | 1414590.54 |
|    6    | 20-SEP-2023 | 10000000 | 5.0  | 506944.44 | 0.8412 | 426433.43 | 1841023.97 |
|    7    | 20-SEP-2024 | 10000000 |

In [17]:
swap.pv01(value_dt, discount_curve)

5.272690754192439

In [18]:
swap.swap_rate(value_dt, discount_curve)

0.04516388971357023

## Including a final exchange of principal

In [19]:
swap.value(settle_dt, discount_curve, discount_curve)

219755.211551575

In [20]:
swap.print_float_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS VALUATION:
+---------+-------------+----------+--------+-----------+--------+-----------+------------+
| PAY_NUM |    PAY_dt   | NOTIONAL |  IBOR  |    PMNT   |   DF   |     PV    |   CUM_PV   |
+---------+-------------+----------+--------+-----------+--------+-----------+------------+
|    1    | 20-SEP-2018 | 10000000 |  0.0   |    0.0    |  0.0   |    0.0    |    0.0     |
|    2    | 20-MAR-2019 | 10000000 |  0.0   |    0.0    |  0.0   |    0.0    |    0.0     |
|    3    | 20-SEP-2019 | 10000000 |  0.0   |    0.0    |  0.0   |    0.0    |    0.0     |
|    4    | 20-MAR-2020 | 10000000 |  0.0   |    0.0    |  0.0   |    0.0    |    0.0     |
|    5    | 21-SEP-2020 | 10000000 | 5.0007 | 256979.26 | 0.9753 | 250643.82 | 250643.82  |
|    6    | 22-MAR-2021 | 10000000 | 4.9997 | 252764.41 | 0.9517 | 240545.23 | 491189.05  |
|    7    |

As expected we see the final PV of the floating Leg is par (same index and discounting curve).

In [21]:
swap.print_fixed_leg_pv()

START DATE: 20-JUN-2018
MATURITY DATE: 22-SEP-2025
COUPON (%): 5.0
FREQUENCY: FrequencyTypes.ANNUAL
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS VALUATION:
+---------+-------------+----------+------+-----------+--------+-----------+------------+
| PAY_NUM |    PAY_dt   | NOTIONAL | RATE |    PMNT   |   DF   |     PV    |   CUM_PV   |
+---------+-------------+----------+------+-----------+--------+-----------+------------+
|    1    | 20-SEP-2018 | 10000000 | 5.0  | 127777.78 |  0.0   |    0.0    |    0.0     |
|    2    | 20-SEP-2019 | 10000000 | 5.0  | 506944.44 |  0.0   |    0.0    |    0.0     |
|    3    | 21-SEP-2020 | 10000000 | 5.0  | 509722.22 | 0.9753 | 497155.79 | 497155.79  |
|    4    | 20-SEP-2021 | 10000000 | 5.0  | 505555.56 | 0.9285 | 469413.12 | 966568.91  |
|    5    | 20-SEP-2022 | 10000000 | 5.0  | 506944.44 | 0.8838 | 448021.63 | 1414590.54 |
|    6    | 20-SEP-2023 | 10000000 | 5.0  | 506944.44 | 0.8412 | 426433.43 | 1841023.97 |
|    7    | 20-SEP-2024 | 10000000 |

In [23]:
swap.fixed_leg.value(settle_dt, discount_curve)

2636345.37709622

In [24]:
swap.float_leg.value(settle_dt, discount_curve, discount_curve)

-2416590.165544645

Copyright (c) 2020 Dominic O'Kane